In [ ]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm
import torch
from datasets import load_dataset
from itertools import islice
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration, BartTokenizerFast


In [ ]:
dataset = load_dataset("trivia_qa", "unfiltered", split="train", streaming=True) 
dataset = list(islice(dataset, 0, 1000)) # Limit to 1000 samples when space is limited

In [ ]:
retriever = RagRetriever.from_pretrained(
    "facebook/rag-token-nq",
    index_name="exact",  # FAISS index trained on DPR-wiki passages
    use_dummy_dataset=True  # loads built-in Wikipedia index
)

model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)


In [ ]:
rag_tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")   
generator_tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-large")


In [ ]:
# Prepare dataset
def process_example(example):
    # Tokenize input (question)
    input_encodings = rag_tokenizer(example['question'], truncation=True, padding="max_length", max_length=512)

    # Use just the "value" field of the answer if it exists
    if isinstance(example['answer'], dict) and 'value' in example['answer']:
        answer_text = example['answer']['value']
    else:
        answer_text = "No answer provided"

    # Tokenize answer (target)
    target_encodings = generator_tokenizer(answer_text, truncation=True, padding="max_length", max_length=128)

    # Return tokenized input and target
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    



In [ ]:
def custom_collate_fn(batch):
    # Collate the batch by padding the sequences to the max length in the batch
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]
    

    # Pad sequences to the max length in each batch (or use a fixed size)
    input_ids_padded = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq) for seq in input_ids], batch_first=True, padding_value=0)
    attention_mask_padded = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq) for seq in attention_mask], batch_first=True, padding_value=0)
    labels_padded = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq) for seq in labels], batch_first=True, padding_value=-100)

    return {
        'input_ids': input_ids_padded,
        'attention_mask': attention_mask_padded,
        'labels': labels_padded,
    }



In [ ]:
#HYPERPARAMETERS
BATCH_SIZE = 4
EPOCHS = 1
accumulation_steps = 8  # Accumulate gradients over 8 steps

# Load dataset and process it
processed_dataset = [process_example(example) for example in dataset]
train_dataloader = DataLoader(processed_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn, num_workers=4, pin_memory=True)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
for epoch in range(EPOCHS):  
    model.train()
    total_loss = 0
    optimizer.zero_grad()  
    
    # Loop over batches
    for i, batch in enumerate(tqdm(train_dataloader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Compute embeddings for the questions
        with torch.no_grad():
            output = model.question_encoder(input_ids=input_ids, attention_mask=attention_mask)
            last_hidden_state = output[0]  
            question_hidden_states = last_hidden_state.cpu().numpy()  

        # 🔁 Multihop change: Retrieve top-k documents and concatenate them into a single context
        n_docs = 5  # Number of documents per hop
        n_hops = 2  # Number of hops (i.e., chained sets of documents)
        all_contexts = []

        # Retrieve contexts for multiple hops
        question_hidden_states_hop = question_hidden_states
        for hop in range(n_hops):
            _, _, doc_dicts = retriever.retrieve(question_hidden_states_hop, n_docs=n_docs)
            hop_contexts = [doc["text"] for doc in doc_dicts]
            hop_contexts = [item for sublist in hop_contexts for item in sublist]
            all_contexts.append(hop_contexts)

            # For the next hop, re-encode the concatenated docs from this hop as the new query
            flat_hop_contexts = [" ".join(docs) for docs in hop_contexts]
            hop_encodings = rag_tokenizer(flat_hop_contexts, return_tensors="pt", padding=True, truncation=True, max_length=512)
            with torch.no_grad():
                hop_output = model.question_encoder(input_ids=hop_encodings['input_ids'].to(device),
                                                    attention_mask=hop_encodings['attention_mask'].to(device))
                question_hidden_states_hop = hop_output[0].cpu().numpy()

        # Concatenate all retrieved documents from both hops
        combined_contexts = [" ".join(docs) for docs in all_contexts]

        context_encodings = generator_tokenizer.batch_encode_plus(
            combined_contexts,
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt"
        )

        context_input_ids = context_encodings['input_ids'].to(device).unsqueeze(1)  # shape: [B, 1, seq_len]
        context_attention_mask = context_encodings['attention_mask'].to(device).unsqueeze(1)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask
        )

        loss = outputs.loss.mean()

        loss = loss / accumulation_steps  
        
        # Backward pass
        loss.backward()

        # Gradient accumulation
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {total_loss / len(train_dataloader)}")

# === Save Model ===
model.save_pretrained("rag_model_finetuned_multihop")
